In [ ]:
! pip install transformers==4.50 -U
! pip install datasets
! pip install scipy sklearn
! pip install evaluate

In [ ]:
import random
import numpy as np
import torch
import os
import transformers

def set_seed(seed: int) -> None:
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  transformers.set_seed(seed)

  # Ensure deterministic behavior
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  # Set a fixed value for the hash seed
  os.environ['PYTHONHASHSEED'] = str(seed)

  print(f'Random seed set as {seed}')


In [ ]:
set_seed(42) #1,5,42

Random seed set as 42


## Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
path = "/content/gdrive/MyDrive/POLINE/"

Mounted at /content/gdrive


In [ ]:
to_classify = 'jif'
language = 'sv'
context = True
version = language + '_context.pkl' if context else language + '.pkl'

# multilingual model
#model_checkpoint = 'google-bert/bert-base-multilingual-cased'

# italian
#model_checkpoint = "dlicari/Italian-Legal-BERT"
#model_checkpoint = "dbmdz/bert-base-italian-cased"

# bulgarian
#model_checkpoint = "DeepPavlov/bert-base-bg-cs-pl-ru-cased"

# swedish
model_checkpoint = "KB/bert-base-swedish-cased"


batch_size = 16
lr = 2e-5
num_epochs = 10
early_stop = 'f1'


In [ ]:
import pandas as pd
df = pd.read_pickle(path + version)

if context:
  df.rename(columns={"text": "sentence"}, inplace=True)
  df.rename(columns={"context": "sentence2"}, inplace=True)
else:
  df.rename(columns={"text": "sentence"}, inplace=True)

labels_list = df[to_classify].dropna().unique()
num_labels = len(labels_list)
label2id = {}
for i, lab in enumerate(labels_list):
  label2id[lab] = i

new_col = [label2id[x] for x in list(df[to_classify])]
df['label'] = new_col

id2label = {val: key for key, val in label2id.items()}

df = df[['sentence', 'sentence2', 'label', 'split']] if context else df[['sentence', 'label', 'split']]


In [ ]:
id2label

{0: 'no', 1: 'yes'}

In [ ]:
from datasets import DatasetDict, Dataset
#from evaluate import load

In [ ]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(df[df.split == 'train']),
    "validation": Dataset.from_pandas(df[df.split == 'validation']),
    "test": Dataset.from_pandas(df[df.split == 'test'])
    })

In [ ]:
dataset["train"][0]

{'sentence': 'HFD 2013 ref 67 \nFråga om mervärdesskatt vid utförande av estetiska operationer och behandlingar.',
 'sentence2': ' HFD 2013 ref 67 \nFråga om mervärdesskatt vid utförande av estetiska operationer och behandlingar. Lagrum: \n3 kap. 4 § och 5 § mervärdesskattelagen (1994:200); artikel 132.1 b och c i direktiv 2006/112/EG \n \n \nPFC Clinic AB:s (bolaget) verksamhet bestod av medicinska tjänster i form av estetiska operationer och behandlingar.',
 'label': 0,
 'split': 'train',
 '__index_level_0__': 2802}

In [ ]:
import datasets
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,sentence,sentence2,label,split,__index_level_0__
0,En prövning ska därför göras av om omsättningen av dessa tjänster kan anses omfattad av det undantag från skattplikt som gäller för idrottstjänster i ML och dess motsvarighet i mervärdesskattedirektivet. -,Detta får även anses gälla för tävlingsverksamheten. En prövning ska därför göras av om omsättningen av dessa tjänster kan anses omfattad av det undantag från skattplikt som gäller för idrottstjänster i ML och dess motsvarighet i mervärdesskattedirektivet. - Enligt 3 kap. 11 a § ML första stycket undantas från skatteplikt bl.a. omsättning av tjänster varigenom någon bereds tillfälle att utöva idrottslig verksamhet samt tjänster som har omedelbart samband med utövandet av den idrottsliga verksamheten om dessa tjänster omsätts av den som tillhandahåller den idrottsliga verksamheten.,0,train,8040
1,I SoL behandlas socialnämndens respektive kommunens ansvar att tillhandahålla vård i form av HVB (5 kap. och 6 kap.) och finns bestämmelser om hur vården ska bedrivas (6 kap. 4 § och 11 kap. 3 §).,Vid bedömningen av Magelungens respektive O.L:s karaktär av erkänt socialt organ är vissa bestämmelser i SoL och i lagen (1998:531) om yrkesverksamhet på hälso- och sjukvårdens område av intresse. I SoL behandlas socialnämndens respektive kommunens ansvar att tillhandahålla vård i form av HVB (5 kap. och 6 kap.) och finns bestämmelser om hur vården ska bedrivas (6 kap. 4 § och 11 kap. 3 §). Stöd- och hjälpinsatser av behandlingskaraktär medför inte kostnadsansvar för den enskilde vårdtagaren.,0,train,3714
2,Enligt 3 kap. 9 § första stycket undantas från skatteplikt omsättning av bank- och finansieringstjänster samt sådan omsättning som utgör värdepappershandel eller därmed jämförlig verksamhet.,"Från skatteplikt undantas, med vissa nu inte aktuella begränsningar, omsättning av fastigheter samt överlåtelse och upplåtelse av bostadsrätter m.fl. rättigheter till fastigheter (3 kap. 2 § ML). - Enligt 3 kap. 9 § första stycket undantas från skatteplikt omsättning av bank- och finansieringstjänster samt sådan omsättning som utgör värdepappershandel eller därmed jämförlig verksamhet. Med värdepappershandel förstås enligt tredje stycket 1 samma paragraf omsättning och förmedling av aktier, andra andelar och fordringar, oavsett om de representeras av värdepapper eller inte. -",0,train,3006
3,i de situationer som regleras i 7 § LMK.,"Enligt den lagen lämnades dock ersättning endast vid upphandling och bidragsgivning, dvs. i de situationer som regleras i 7 § LMK. För verksamhet som bedrevs i egen regi gällde i stället att kommuner och landsting hade rätt till avdrag för ingående skatt vid mervärdesbeskattningen även på områden som var undantagna från skatteplikt.",0,train,8876
4,Ansvar för medicinska frågor - Behandling av medicinska tillstånd hos patienter och tidvis personal - Bolagets uppdragsgivare är oftast kommunernas socialförvaltningar men även andra uppdragsgivare såsom Kriminalvårdsstyrelsen eller t.ex. privata företag förekommer. -,Hans verksamhet består av: Utvärdering och bedömning av patienter på besök - Undersökning och diagnostik av såväl nya som redan intagna patienter - Kontinuerlig bedömning av patienter i sjukvården - Ansvar för patienter inom sjukvården på behandlingshemmet. - Ansvar för medicinska frågor - Behandling av medicinska tillstånd hos patienter och tidvis personal - Bolagets uppdragsgivare är oftast kommunernas socialförvaltningar men även andra uppdragsgivare såsom Kriminalvårdsstyrelsen eller t.ex. privata företag förekommer. - När en kommun är uppdragsgivare har kommunens socialförvaltning i enlighet med socialtjänstlagen fattat ett beslut om vård avseende den enskilde klienten. -,0,train,5055
5,"Nyström, Odéen, Ohlson, Peterson, Rabe): Förhandsbesked.","- Skatterättsnämnden (2002-09-20, Wingren, ordf., Nyström, Odéen, Ohlson, Peterson, Rabe): Förhandsbesked. Publikationen utgör inte ett sådant periodiskt medlemsblad som avses i 3 kap. 13 § ML. -",0,train,4808
6,Mot

## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
  if context:
    return tokenizer(examples['sentence'], examples['sentence2'], truncation=True, padding=True)
  else:
    return tokenizer(examples['sentence'], truncation=True, padding=True)

In [ ]:
pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/6371 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1531 [00:00<?, ? examples/s]

Map:   0%|          | 0/1271 [00:00<?, ? examples/s]

Columns added by tokenizer: ['token_type_ids', 'input_ids', 'attention_mask']


## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from sklearn.metrics import classification_report

#########################################################

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
)
model_name = model_checkpoint.split("/")[-1]

In [ ]:
from evaluate import load

In [ ]:
args = TrainingArguments(
    f"{model_name}-finetuned-{to_classify}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none"
)

metric = load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
trainer.evaluate()

predictions, labels, metrics = trainer.predict(encoded_dataset["test"])
predicted_classes = np.argmax(predictions, axis=-1)

y_true = [id2label[x] for x in labels]
y_pred = [id2label[x] for x in predicted_classes]
print()
print(classification_report(y_true, y_pred, digits=4))
print()
print(y_true)
print(y_pred)
print()


In [ ]:
model_path = "/model_" + language

In [ ]:
model.save_pretrained(path+ model_path)
tokenizer.save_pretrained(path+model_path)